In [1]:
import pyfaidx
import numpy as np
import pyBigWig
from modisco.visualization import viz_sequence
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "/mnt/lab_data2/anusri/chrombpnet/src/helpers/preprocessing/analysis/")
import one_hot
import pandas as pd

def get_pwm_bg(seqs, cnts, new_seqs, pwm_width=24):
    '''
    Arguments::
        seqs: An input array of shape L x alphabet
        cnts: An input array of shape L
    
    Returns:
        motif: PPM (Position Probability Matrix) dimensions of length x alphabet.
               Entries along the alphabet axis sum to 1.
        bg: The background base frequencies
    '''
    for i in range(pwm_width//2,cnts.shape[0]-pwm_width//2):
        if cnts[i] > 0:
            new_seqs.append(seqs[i-pwm_width//2:i+pwm_width//2,:]*cnts[i])
    return new_seqs


In [2]:
# access files
genome="/mnt/lab_data2/anusri/chrombpnet/reference/hg38.genome.fa"
#bigwig="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/GM12878/data/GM12878_unstranded.bw"
#bigwig="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/NAKED/data/NAKED_unstranded.bw"
#bigwig="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/HEPG2/data/HEPG2_unstranded.bw"
#31
#bigwig="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/IMR90/data/IMR90_unstranded.bw"
#32
#bigwig="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/K562/data/K562_unstranded.bw"
#33
bigwig="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/H1ESC/data/H1ESC_unstranded.bw"

bed="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/GM12878/negatives_data/negatives_with_summit.bed"
#ppm_dir="bias_motifs_invivo"
#ppm_dir="bias_motifs/ATAC/"
ppm_dir='bias_motifs_transfer_hep'

hg38 = pyfaidx.Fasta(genome)
bw = pyBigWig.open(bigwig) 
bedregions=pd.read_csv(bed,sep="\t",header=None)
print(bedregions.head())
new_seqs = []

for i,r in bedregions.iterrows():
    chri=r[0]
    start=r[1]+r[9]-500
    end=r[1]+r[9]+500
    # fetch values in the given chromsome and for the given chromsome region
    seq = str(hg38[chri][start:end])
    one_hot_seq = one_hot.dna_to_one_hot(seq).squeeze()
    bigwig_vals = np.nan_to_num(bw.values(chri,start,end ))
    get_pwm_bg(one_hot_seq, bigwig_vals, new_seqs, pwm_width=24)
    
motif = np.sum(new_seqs, axis=0)
motif = motif/np.sum(motif, axis=-1, keepdims=True)
bg = np.sum(np.sum(new_seqs, axis=0), axis=0)
bg = bg/sum(bg)


      0          1          2  3  4  5  6  7  8     9
0  chr3  105204000  105206114  .  .  .  .  .  .  1057
1  chr3  146462000  146464114  .  .  .  .  .  .  1057
2  chr1   77114000   77116114  .  .  .  .  .  .  1057
3  chr1  116329000  116331114  .  .  .  .  .  .  1057
4  chr1  240701000  240703114  .  .  .  .  .  .  1057


In [3]:
print(motif.shape)

(24, 4)


In [4]:

output = viz_sequence.ic_scale(motif, background=bg)

In [5]:
output.shape

(24, 4)

In [6]:
key="0_33"
mode="profile"
f = open(os.path.join(ppm_dir,mode+"_"+key+"_tn5.pfm"),"w")
np.savetxt(f, output, fmt='%f')
f.close()

NameError: name 'os' is not defined